Step 1: Import Libraries

In [1]:
! pip install pandas numpy scikit-learn xgboost


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Python312\python.exe -m pip install --upgrade pip


In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import StackingClassifier


Step 2: Load & Preprocess the Data


In [3]:
# Load dataset
df = pd.read_csv("dataset.csv")  # Update path if needed

# Drop unnecessary columns
df = df.drop(columns=["Unnamed: 0"], errors="ignore")

# Remove duplicates
df = df.drop_duplicates()

# Separate features & target variable
X = df.drop(columns=["label"])
y = df["label"]

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training & testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


Step 3: Train Base Models

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer # Import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import StackingClassifier

# Load dataset
df = pd.read_csv("dataset.csv")  # Update path if needed

# Drop unnecessary columns
df = df.drop(columns=["Unnamed: 0"], errors="ignore")

# Remove duplicates
df = df.drop_duplicates()

# Separate features & target variable
X = df.drop(columns=["label"])
y = df["label"]

# Impute missing values with the mean of each column
imputer = SimpleImputer(strategy='mean') # Create an imputer instance
X = imputer.fit_transform(X) # Impute missing values in X

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training & testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Initialize models
log_reg = LogisticRegression(max_iter=1000, random_state=42)
rf = RandomForestClassifier(n_estimators=100, random_state=42)
xgb = XGBClassifier(n_estimators=100, use_label_encoder=False, eval_metric="logloss", random_state=42)

# Train models
log_reg.fit(X_train, y_train)
rf.fit(X_train, y_train)
xgb.fit(X_train, y_train)

# Get predictions
log_reg_pred = log_reg.predict(X_test)
rf_pred = rf.predict(X_test)
xgb_pred = xgb.predict(X_test)

# Evaluate base models
log_reg_acc = accuracy_score(y_test, log_reg_pred)
rf_acc = accuracy_score(y_test, rf_pred)
xgb_acc = accuracy_score(y_test, xgb_pred)

print(f"Logistic Regression Accuracy: {log_reg_acc:.4f}")
print(f"Random Forest Accuracy: {rf_acc:.4f}")
print(f"XGBoost Accuracy: {xgb_acc:.4f}")

C:\Users\ykuma\AppData\Roaming\Python\Python312\site-packages\xgboost\training.py:183: UserWarning: [23:27:06] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Logistic Regression Accuracy: 0.9654
Random Forest Accuracy: 0.9798
XGBoost Accuracy: 0.9776


Step 4: Create the Stacking Model

In [5]:
# Define base models
base_models = [
    ("log_reg", log_reg),
    ("rf", rf),
    ("xgb", xgb)
]

# Define stacking classifier with Logistic Regression as meta-model
stacked_model = StackingClassifier(estimators=base_models, final_estimator=LogisticRegression(), cv=5)

# Train the stacking model
stacked_model.fit(X_train, y_train)

# Get predictions from stacked model
stacked_pred = stacked_model.predict(X_test)

# Evaluate stacking model
stacked_acc = accuracy_score(y_test, stacked_pred)

print(f"Stacking Model Accuracy: {stacked_acc:.4f}")


C:\Users\ykuma\AppData\Roaming\Python\Python312\site-packages\xgboost\training.py:183: UserWarning: [23:28:33] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\ykuma\AppData\Roaming\Python\Python312\site-packages\xgboost\training.py:183: UserWarning: [23:35:06] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\ykuma\AppData\Roaming\Python\Python312\site-packages\xgboost\training.py:183: UserWarning: [23:35:12] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\ykuma\AppData\Roaming\Python\Python312\site-packages\xgboost\training.py:183: UserWarning: [23:35:18] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.

Stacking Model Accuracy: 0.9810


Final Step: Compare Model Accuracies


In [6]:
print("\nModel Performance Comparison:")
print(f"Logistic Regression Accuracy: {log_reg_acc:.4f}")
print(f"Random Forest Accuracy: {rf_acc:.4f}")
print(f"XGBoost Accuracy: {xgb_acc:.4f}")
print(f"Stacking Model Accuracy: {stacked_acc:.4f} ")



Model Performance Comparison:
Logistic Regression Accuracy: 0.9654
Random Forest Accuracy: 0.9798
XGBoost Accuracy: 0.9776
Stacking Model Accuracy: 0.9810 
